In [2]:
import pickle
import json 
from collections import defaultdict

In [3]:
# look at the saved mmlu task 

with open("/home/rahul/code/clean/lolcats/mmlu.pkl", 'rb') as f:
    data = pickle.load(f)
task_names = set()
for _, i in enumerate(data):
    # print(i[0])
    if _ == 400:
        break
    task_names.add(i[1])
print(task_names)


{'hendrycksTest-abstract_algebra'}


In [4]:
# this code sanity checks my fixes

# compare the two implementations
import os
path = "/home/rahul/code/clean/lolcats/preds_so_far/"
files = os.listdir(path)
files = [f"{path}/{f}" for f in os.listdir(path) if "0.pkl" in f and "v2" not in f and "v3" not in f]

# checkpoint 1670 red pajama cria
new_files = f"{path}/v2_mmlu_predictions_0.pkl"
print(new_files)
with open(new_files, 'rb') as f:
    predictions_new = pickle.load(f)

# checkpoint 3500 red pajama cria
new_files = f"{path}/v3_mmlu_predictions_0.pkl"
print(new_files)
with open(new_files, 'rb') as f:
    predictions_new_v3 = pickle.load(f)
# additional preds
more_files = [f for f in os.listdir("/home/rahul/code/clean/lolcats/") if "v3" in f]
step2preds = {}
for f in more_files:
    step = f.split("v3_")[-1].split("_")[0]
    step = int(step)
    print(f"Step: {step}")
    with open(f"/home/rahul/code/clean/lolcats/{f}", 'rb') as f:
        preds = pickle.load(f)
    step2preds[step] = preds

from collections import defaultdict
taskname2correct = defaultdict(list)
taskname2correct_new = defaultdict(list)
taskname2correct_new_v3 = defaultdict(list)
for i, f_ in enumerate(files):
    with open(f_, 'rb') as f:
        predictions = pickle.load(f)
    for idx, query in enumerate(predictions.keys()):
        ans_pred = predictions[query]
        task_name = " ".join(query.split("-")[0:2])
        taskname2correct[task_name].append(ans_pred)
        if idx < len(predictions_new):
            new_pred = predictions_new[idx]
            taskname2correct_new[task_name].append(new_pred)
        if idx < len(predictions_new_v3): 
            new_pred_v3 = predictions_new_v3[idx]
            taskname2correct_new_v3[task_name].append(new_pred_v3)

for i, (task, corrects) in enumerate(taskname2correct.items()):
    print(task, f"{sum(corrects)/len(corrects)}, {len(corrects)}")
    if task in taskname2correct_new:
        corrects_new = taskname2correct_new[task]
        print(f"-> {i} V2: {task}", f"{sum(corrects_new)/len(corrects_new)}, {len(corrects_new)}")
    if task in taskname2correct_new_v3:
        corrects_new_v3 = taskname2correct_new_v3[task]
        print(f"-> {i} V3: {task}", f"{sum(corrects_new_v3)/len(corrects_new_v3)}, {len(corrects_new_v3)}")
    print()

corrects_new = []
for task, corrects_ in taskname2correct_new.items():
    corrects_new.extend(corrects_)
print("\n New Implementation V2: ")
print(sum(corrects_new)/len(corrects_new))

corrects_new_v3 = []
for task, corrects_ in taskname2correct_new_v3.items():
    corrects_new_v3.extend(corrects_)
print("\n New Implementation V3: ")
print(sum(corrects_new_v3)/len(corrects_new_v3))


/home/rahul/code/clean/lolcats/preds_so_far//v2_mmlu_predictions_0.pkl


/scratch/rahul/miniforge3/envs/serve/lib/python3.12/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


/home/rahul/code/clean/lolcats/preds_so_far//v3_mmlu_predictions_0.pkl


ValueError: invalid literal for int() with base 10: 'steps'

In [35]:
# compare the two implementations
import os
path = "/home/rahul/code/clean/lolcats/preds_so_far/"

# checkpoint 1670 red pajama cria
new_files = f"{path}/v2_mmlu_predictions_0.pkl"
print(new_files)
with open(new_files, 'rb') as f:
    predictions_1 = pickle.load(f)

# checkpoint 3500 red pajama cria
new_files = f"{path}/v3_mmlu_predictions_0.pkl"
print(new_files)
with open(new_files, 'rb') as f:
    predictions_2 = pickle.load(f)
print(f"Initial preds: {len(predictions_2)}")
more_files = [f for f in os.listdir("/home/rahul/code/clean/lolcats/steps_more_v3/") if "v3" in f and "00_mmlu" in f]
max_int  = 0
max_file = ''
for f in more_files:
    step = f.split("v3_")[-1].split("_")[0]
    step = int(step)
    if step > max_int: 
        max_int = step
        max_file = f
print(f"Max step: {max_int}")
with open(f"/home/rahul/code/clean/lolcats/steps_more_v3/{max_file}", 'rb') as f:
    preds = pickle.load(f)
predictions_2.extend(preds)
print(f"Additional preds: {len(preds)}\n")

# get accuracies
taskname2correct_1 = defaultdict(list)
taskname2correct_2 = defaultdict(list)
for i, p2 in enumerate(predictions_2):
    if i % 100 == 0:
        task_name = f"task_{i}"
    taskname2correct_2[task_name].append(p2.item())
    if i < len(predictions_1):
        p1 = predictions_1[i]
        taskname2correct_1[task_name].append(p1.item()) 

# average by task
for task, corrects in taskname2correct_1.items():
    print(f"->", task, f"%.3f (%d)", (sum(corrects)/len(corrects), len(corrects)))
    if task in taskname2correct_2:
        corrects_2 = taskname2correct_2[task]
        print(f"->", task, f"%.3f (%d)", (sum(corrects_2)/len(corrects_2), len(corrects_2)))

# overall accuracy
corrects = []
for task, corrects_ in taskname2correct_1.items():
    corrects.extend(corrects_)
print("\n Initial Implementation: ")
print(sum(corrects)/len(corrects))
print(len(corrects))

corrects_2 = []
for task, corrects_ in taskname2correct_2.items():
    corrects_2.extend(corrects_)
print("\n New Implementation: ")
print(sum(corrects_2)/len(corrects_2))
print(len(corrects_2))

/home/rahul/code/clean/lolcats/preds_so_far//v2_mmlu_predictions_0.pkl
/home/rahul/code/clean/lolcats/preds_so_far//v3_mmlu_predictions_0.pkl
Initial preds: 3201
Max step: 14000
Additional preds: 10800

-> task_0 %.3f (%d) (0.52, 100)
-> task_0 %.3f (%d) (0.46, 100)
-> task_100 %.3f (%d) (0.78, 100)
-> task_100 %.3f (%d) (0.78, 100)
-> task_200 %.3f (%d) (0.8, 100)
-> task_200 %.3f (%d) (0.85, 100)
-> task_300 %.3f (%d) (0.85, 100)
-> task_300 %.3f (%d) (0.89, 100)
-> task_400 %.3f (%d) (0.61, 100)
-> task_400 %.3f (%d) (0.76, 100)
-> task_500 %.3f (%d) (0.79, 100)
-> task_500 %.3f (%d) (0.81, 100)
-> task_600 %.3f (%d) (0.73, 100)
-> task_600 %.3f (%d) (0.81, 100)
-> task_700 %.3f (%d) (0.78, 100)
-> task_700 %.3f (%d) (0.88, 100)
-> task_800 %.3f (%d) (0.89, 100)
-> task_800 %.3f (%d) (0.93, 100)
-> task_900 %.3f (%d) (0.5, 100)
-> task_900 %.3f (%d) (0.63, 100)
-> task_1000 %.3f (%d) (0.67, 100)
-> task_1000 %.3f (%d) (0.71, 100)
-> task_1100 %.3f (%d) (0.5, 100)
-> task_1100 %.3f (

In [25]:
# compare the two implementations
import os
path = "/home/rahul/code/clean/lolcats/preds_so_far/"

# checkpoint 3500 red pajama cria
new_files = f"/home/rahul/code/clean/lolcats/preds_so_far//v3_mmlu_predictions_0.pkl"
with open(new_files, 'rb') as f:
    predictions_2 = pickle.load(f)
print(f"Initial preds: {len(predictions_2)}")

with open(f"/home/rahul/code/clean/lolcats/steps_more_v3/v3_14041_mmlu_predictions_0.pkl", 'rb') as f:
    preds = pickle.load(f)
predictions_2.extend(preds)
print(f"Additional preds: {len(preds)}\n")

# overwrites
# with open(f"/home/rahul/code/clean/lolcats/steps_more_v4/v4_14041_mmlu_predictions_0.pkl", 'rb') as f:
#     preds_overwrites = pickle.load(f)
# with open(f"/home/rahul/code/clean/lolcats/steps_more_v5/v5_14041_mmlu_predictions_0.pkl", 'rb') as f:
#     preds_overwrites_2 = pickle.load(f)
# preds_overwrites.update(preds_overwrites_2)
preds_overwrites = {}

# get accuracies
overwrites = 0
taskname2correct_2 = defaultdict(list)
for i, p2 in enumerate(predictions_2):
    if i in preds_overwrites:
        p2 = preds_overwrites[i]
        overwrites += 1
    if i % 100 == 0:
        task_name = f"task_{i}"
    taskname2correct_2[task_name].append(p2.item()) 
print(f"Overwrites: {overwrites}\n")

# average by task
corrects_2 = []
for task, corrects_ in taskname2correct_2.items(): corrects_2.extend(corrects_)
print("New Implementation: ") 
print(sum(corrects_2)/len(corrects_2))
print((sum(corrects_2) - 166)/len(corrects_2))
print(len(corrects_2))

Initial preds: 3201
Additional preds: 10841

Overwrites: 0

New Implementation: 
0.7539524284289987
0.7421307506053268
14042


In [53]:
# compare the two implementations

path = "/home/simarora/code/lolcats/reference_70b_outputs/hendrycksTest-abstract_algebra_write_out_info.json"
with open(path, 'r') as f:
    reference = json.load(f)

path = "/home/simarora/code/lolcats/mmlu_predictions_0.pkl"
with open(path, 'rb') as f:
    predictions = pickle.load(f)

avg_ref = []
avg_pred = []
for i, j in zip(reference, predictions.keys()):
    ans_ref = i['acc']
    ans_pred = predictions[j]
    avg_ref.append(float(ans_ref))
    avg_pred.append(float(ans_pred))
    if float(ans_ref) != float(ans_pred):
        print(i['prompt_0'].split("\n\n")[-1].split("\nA")[0])
        print(f"--> {int(float(i['acc']))}")
        print(j.split("\nA")[0].split("-")[2])
        print(f"--> {int(predictions[j])}")
        print("--------------------")

print(sum(avg_ref)/len(avg_ref))
print(sum(avg_pred)/len(avg_pred))


Let p = (1, 2, 5, 4)(2, 3) in S_5 . Find the index of <p> in S_5.
--> 1
Let p = (1, 2, 5, 4)(2, 3) in S_5 . Find the index of <p> in S_5.
--> 0
--------------------
(Z,*) is a group with a*b = a+b+1 for all a, b in Z. The inverse of a is
--> 1
(Z,*) is a group with a*b = a+b+1 for all a, b in Z. The inverse of a is
--> 0
--------------------
Find the characteristic of the ring Z_3 x Z_3.
--> 0
Find the characteristic of the ring Z_3 x Z_3.
--> 1
--------------------
Statement 1 | For any two groups G and G', there exists a homomorphism of G into G'. Statement 2 | Every homomorphism is a one-to-one map.
--> 1
Statement 1 | For any two groups G and G', there exists a homomorphism of G into G'. Statement 2 | Every homomorphism is a one
--> 0
--------------------
Statement 1 | If a R is an integral domain, then R[x] is an integral domain. Statement 2 | If R is a ring and f(x) and g(x) are in R[x], then deg (f(x)g(x)) = deg f(x) + deg g(x).
--> 1
Statement 1 | If a R is an integral domain, 